### Diversity of prophages in KASPAH

In [29]:
# jupyter setup
%reload_ext autoreload
%autoreload 2

# import modules
import warnings
import pandas as pd
from pathlib import Path
from Bio import Phylo
from scripts.utils import preprocessing, coGRR, get_phariants, run_mashtree
from scripts.utils import tree2clades, between_phariants_easyfig
from scripts.utils import stGRR, GRR
warnings.filterwarnings('ignore')

In [30]:
# paths
work_dir = '/home/MCB/jkoszucki/phagedb/PHAGES-DB'
animm_dir = '/home/MCB/jkoszucki/Code/ANImm'
phrogs_annot_table = '/home/MCB/jkoszucki/Code/phage-diversity/other/upgraded_phrog_annot_v3.tsv'
# font_path='other/arial.ttf' 

input_dir = '/home/MCB/jkoszucki/phagedb'
inphared_dir = Path(input_dir, 'INPHARED-DB-1Aug2022/INPHARED-DB-1Aug2022-KLEBSIELLA')
prophages_dir = Path(input_dir, 'PROPHAGES-DB-1Aug2022/prophages')


phagedb_dir = Path(work_dir, '0_phagedb')
annot_input = Path(phagedb_dir, 'annot_input.txt')
metadata_table = Path(phagedb_dir, 'phages.tsv')
genbank_dir = Path(phagedb_dir, 'split_records/genbank')


#coGRR
coGRR_dir = Path(work_dir, '1_coGRR')

coGRR_animm_dir = Path(coGRR_dir, '1_ANImm')
wgrr = Path(coGRR_animm_dir, 'wgrr.csv')
phariants = Path(coGRR_animm_dir, 'phariants.tsv')

tree_dir = Path(coGRR_dir, '2_mashtree')
tree_path = Path(tree_dir, 'tree.newick')
clades = Path(tree_dir, 'clades.tsv')

#stGRR
stGRR_dir = Path(work_dir, '2_stGRR')


# GRR
coGRR_table = Path(work_dir, '1_coGRR','1_ANImm', 'wgrr.csv')
stGRR_table = Path(work_dir, '2_stGRR', '2_ANImm', 'wgrr.csv')
GRR_dir = Path(work_dir, '3_GRR')


# params
wgrr_threshold = 0.95
n_clusters = 40 # mashtree clustering

columns2annotate = ['phageID', 'backphlip', 'K_locus', 'ST', 'genetic_localisation', 'ICTV_Family'] # easyfig annotation
leg_name = 'structural' # easyfig
categs = ['head and packaging', 'connector', 'tail'] # categories of structural proteins

## 0. Preprocessing

In [31]:
### preprocessing of PROPHAGE-DB-1Aug2022 & INPHARED-DB-1Aug2022-KLEBSIELLA & BACKPHLIP
# integrate data

preprocessing(inphared_dir, prophages_dir, phrogs_annot_table, phagedb_dir) 

Loading backphlip from /home/MCB/jkoszucki/backphlip/phages.fasta.bacphlip... Done!
Records saved successfully :) 
Proteins files per phage copied successfully (ANImm input) :)
Done! 1407 phages converted to genbank file.
Functional annotation input tables merged and saved :)
Phage flipped and saved to seperate genbank files :) 
Metadata unified and copied successfully :) 


## 1. complete genome (coGRR)

In [4]:
### calculate wGRR
process = coGRR(animm_dir, phagedb_dir, coGRR_animm_dir)

ANImm already done! To rerun delete folder: /home/MCB/jkoszucki/phagedb/PHAGES-DB/1_coGRR/1_ANImm


In [5]:
### phariants from wGRR
# MCL community detection
phariants_df = get_phariants(wgrr, annot_input, phariants, wgrr_threshold)

Check! In some case I can loose singletons here!
Done! With wGRR treshold = 0.95 we have 964 phage clusters :)

In [6]:
### mashtree
# local machine needs a lots of memory because tree is firstly saved loccaly then copied
cmd = run_mashtree(phagedb_dir, tree_dir)

Just run the command in bash. Problem with conda env AGAIN : / 

source ~/.bashrc; conda activate mashtree; mashtree.pl /home/MCB/jkoszucki/phagedb/PHAGES-DB/0_phagedb/split_records/fasta/* >> /home/MCB/jkoszucki/phagedb/PHAGES-DB/1_coGRR/2_mashtree/tree.newick; conda activate mybase;


In [7]:
### get 'clades' based on mashtree (for visualisation purposes)
# there are size limitations on jpeg files and its easier to look on genomes when they are clustered 
phage_clusters_df = tree2clades(tree_path, phariants, clades, n_clusters, kmeans_show=False)

Mashtree already done! To rerun delete folder: /home/MCB/jkoszucki/phagedb/PHAGES-DB/1_coGRR/2_mashtree/tree.newick


In [5]:
### get easyfig figures between phariants of each representative phage from cluster
# annotated : ) 
results_dir, process = between_phariants_easyfig(work_dir, clades, metadata_table, genbank_dir, leg_name=leg_name, columns2annotate=columns2annotate)

Generating easyfig figures for 40 clusters :) 


FileNotFoundError: [Errno 2] No such file or directory: '/home/MCB/jkoszucki/phagedb/PHAGES-DB/0_phagedb/phages.tsv'

## 2. Structural genome (stGRR)

In [16]:
### get structural clusters
stGRR(animm_dir, phagedb_dir, stGRR_dir, categs)

Prepared input structural proteins :) 
stGRR already done! To rerun ANImm delete folder: /home/MCB/jkoszucki/phagedb/PHAGES-DB/2_stGRR/2_ANImm
Clustering by stGRR... (tresholds: 0.9, 0.8) Done!
Easyfig of stGRR clusters... Done!


## 3. coGRR & stGRR comparison

In [37]:
GRR_df = GRR(coGRR_table, stGRR_table, metadata_table, genbank_dir, 
             GRR_dir, network_grr_tresholds = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1], 
             wgrr_co=0.3 , wgrr_st=0.8, easyfig=False, show=False, force=True)

Running GRR!
Merging tables... Done!
Generating networks... Done!
Generating correlation plot... 
    1. prophages & inphared... Done!
    2. inphared... Done!
